#### This is notebook is my first attempt to browse and visualize the data in the Microsoft Spam Database.

##### Data is from Microsoft Malware Prediction Competition in Kaggle https://www.kaggle.com/c/microsoft-malware-prediction
##### Some of code is borrowed from: https://www.kaggle.com/datark1/malware-prediction-eda

##### Let's load required libraries for ready data and visualizaion.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests 
import time as t
from bs4 import BeautifulSoup

import os
# print(os.listdir("C:/Users/Shahram/Desktop/Microsoft Spam/"))

##### The train.CSV file is almost 4GB. 
* Initially, I could not read the file as a whole so read the data in small chunks to figure out the columns and its content.
* After discussing with Debadyuti, he suggested to define the Types of columns and then use the Pandas read_csv to read all the data.

In [3]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }


train = pd.read_csv("C:/Users/Shahram/Desktop/Microsoft Spam/train.csv", dtype=dtypes)

MemoryError: 

# Ref: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76384#449402

base_url = 'https://www.microsoft.com/en-us/wdsi/definitions/antimalware-definition-release-notes?RequestVersion='
#read the data the delete it then..
#do the same for test set as well..

df = pd.read_csv('train.csv', usecols=['AvSigVersion'])
search_ver = np.asarray(list(set(df['AvSigVersion'].values)))
del df

print(len(search_ver))

saved, cnt = {}, 0

for i in search_ver:
    if cnt%200 == 0:
        print('Done till', cnt, 'out of', len(search_ver))

    page = requests.get(base_url+i)
    soup = BeautifulSoup(page.text, 'html.parser')
    try:
        saved[i] = soup.find(id='availabledate').contents[0][28:]
    except Exception:
        print('Didn\'t Find For', i)
    cnt+= 1
np.save('train_AvSigVersion.npy', saved)


In [ ]:
# IMPORT TIMESTAMP DICTIONARY
datedict = np.load('C:/Users/Shahram/Desktop/Microsoft Spam/AvSigVersionTimestamps.npy', allow_pickle=True)
datedict = datedict[()]
# ADD TIMESTAMPS
train['Date'] = train['AvSigVersion'].map(datedict)
#df_test['Date'] = df_test['AvSigVersion'].map(datedict)

In [ ]:
train.head()

#### Let's see the data and its columns:
* There are almost 8.921 Million records with 83 columns. Memory usage is around 1.9 GB. the columns type is as we defined.

train.info()

train.describe()

#### We will explore the number of unique products and the Engione version in this data sets.

print ("There are", len (train.ProductName.unique().tolist()), "distinct products.", train.ProductName.unique().tolist())
print ("There are", len (train.EngineVersion.unique()), "distinct version of these products.")

### Let's look at distribution of the Devices where a malware has been detected.
#### It is clear that the there is equal distribution of devices with malware detected and device without any malware.

Zero = (train['HasDetections']==0).sum()
One  = (train['HasDetections']==1).sum()
sizes = [Zero,One]
labels = ['Zero','One']
fig1, ax1 = plt.subplots()
plt.title("Check the dsitribution of whether Devices have Detection or not", fontdict=None, loc='center', pad=None, color = "azure")
ax1.pie(sizes, labels = labels, autopct='%1.1f%%',shadow=True, startangle=90)
plt.show()

#### Let's see which columns are binary: 
###### I wrote the code myself but got help from https://www.kaggle.com/datark1/malware-prediction-eda

In [ ]:
BinaryColumns = []
for column in train.select_dtypes (include = ['int8', 'int16', 'int32', 'int16', 'float32', 'float64']):
    if ( train [column].min()) == 0 or (train[column].max() == 1):
        BinaryColumns.append (column)
        #print (column)
print ('There are', len(BinaryColumns),'columns which are Binary.')
BinaryColumns

#### Let's visually look at the percentage of zeros and ones in Digital columns.

fig = plt.figure(figsize=(20,10))
for i, num in zip(BinaryColumns, range(1,20)):
    Zero = (train[i]==0).sum()
    One = (train[i]==1).sum()
    sizes = [Zero,One]
    labels = ['Zero','One']
    ax = fig.add_subplot(4,4,num) #fig1, ax1 = plt.subplots()
    plt.title(i, fontdict=None, loc='center', pad=None, color = "azure")
    ax.pie(sizes, labels = labels, autopct='%1.1f%%',shadow=True, startangle=90)
    ax.legend()

plt.show()

#### We should take a look to see if the columns are correlated.
##### As one can see aside from few columns ( which are in the vicinity of each other, the rest of the data are NOT Correlated.
##### Few columns are negatively correlated. (e.g.: RtpStateBitField - IsSXSPassiveMode)


#### Reference: https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

corr = train.corr()
plt.figure(figsize=(22,12))
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(20, 220, n=200), square=True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

#### Counting the number of Null values in columns and calculating the percentages
#### We can use this conclusion to find out which columns we can drop from our calculations.
#### As one can see, there are 7 columns with more than 50% row of null values.

In [ ]:
NoEmptyRows = []
percentages = []
ColumnType = []

for col in train:
    ColumnType.append(train[col].dtype)
    NoEmptyRows.append(train[col].isnull().sum())
    percentages.append((train[col].isnull().sum()/train.shape[0])*100)

In [ ]:
MissingValueTable = pd.DataFrame ({'Column Name': train.columns, 'Data Type': ColumnType, 
                                   'Number of Empty Rows': NoEmptyRows,'Percentages':percentages})

In [ ]:
MissingValueTable

# Graph Missing Value Percentages
plt.figure(figsize=(20,10))
plt.title("Percentage of Null counts in columns", fontdict=None, loc='center', pad=None)
plt.xticks(rotation='vertical')
plt.bar(x = MissingValueTable['Column Name'], height = MissingValueTable['Percentages'])
plt.show()

### Let's group infected machines by the "platform".
#### Windows 10 is the major infected platform with more than 4 million machines infected.

group = ['Platform', 'HasDetections']
test = train.groupby(group).size().sort_index()

Test = pd.DataFrame (test)

# Graph Missing Value Percentages
plt.figure(figsize=(10,6))
plt.title("Machine counts", fontdict=None, loc='center', pad=None)
plt.xticks(rotation='vertical')
plt.ylabel("Number of machines") #ax.set_ylabel('Probability density')
sns.barplot(x = test.index, y = Test[0])
plt.show()

### Let's group infected machines by Form Factor.

#### It is clear that Notebooks are majorly hit by malwares and then desktops and convertibles stand second and third respectively.

group = ['Census_MDC2FormFactor', 'HasDetections']
test = train.groupby(group).size().sort_index()

Test = pd.DataFrame (test)

# Graph Missing Value Percentages
plt.figure(figsize=(10,6))
plt.title("Machine counts by Form Factor", fontdict=None, loc='center', pad=None)
plt.xticks(rotation='vertical')
plt.ylabel("Number of machines") 
sns.barplot(x = test.index, y = Test[0])
plt.show()

### Let's group infected machines by whether they are a Gamer or not.
#### It is clear that the gamer machine (wehether infected or not) are more than half of the non-gamers. Clearly, among the gamer machine infection is more (probable).

group = ['Wdft_IsGamer', 'HasDetections']
test = train.groupby(group).size().sort_index()

Test = pd.DataFrame (test)

# Graph Missing Value Percentages
plt.figure(figsize=(10,6))
plt.title("Machine counts by whether it is a gaming machine", fontdict=None, loc='center', pad=None)
plt.xticks(rotation='vertical')
plt.ylabel("Number of machines") 
sns.barplot(x = test.index, y = Test[0])
plt.show()

BinaryColumns

CategoryColumns = train.select_dtypes (include = ['category']).columns
# CategoryColumns
BinaryColumns.extend(CategoryColumns)
CategoryColumns
# BinaryColumns

ColumnsExcluded = train.columns.difference(BinaryColumns)

fig = plt.figure(figsize=(20,10))
for i, num in zip(ColumnsExcluded, range(1,30)):
    
    ax = fig.add_subplot(5,6,num) #fig1, ax1 = plt.subplots()
    plt.title(i, fontdict=None, loc='center', pad=None, color = "azure")
    sns.distplot(train[i].dropna())
    #ax.legend()

plt.show()

### Summary:
* We used the data in Microsoft Malware database.
* The data contain around 8.92 million record and 83 column. Fifteen columns are binary. We made a pie chart to see the distributoion of the Binary columns.
* The Target column which is HasDetectiosn is equally distributed.
* We graphed the heatmap of correlation of columns and saw that few columns are correlted. (Two columns are strongly negatively correlated and a few are positivley correlated.)
* we also grahphed the malware infected devices by platform and Form Factor and Windows versio.
* Windos 10 and laptops are the most infected machines.
* We also graphed the histogram of the rmeainning columns to see the distribution of the data.
* Few columns (around 7) have more than 50% of their data mssining (Null value) and hence might be a good idea to ignore them in the data engineering. We ideate that the other columns with less missing values can be filled with mean or median of the column.


### The rest of this note book is related to Feature Engineering.

#### I am going to drop the columns with more than 10% missing vlaues which are around 9 or 10 columns.

In [ ]:
print ("There are ", (MissingValueTable['Percentages']>=10).sum(), "columns with more than 10 % of rows with missing values.")
MissingValueTable [(MissingValueTable['Percentages']>=10)]

In [ ]:
train = train.drop(MissingValueTable [(MissingValueTable['Percentages']>=10)]['Column Name'], axis = 1)

In [ ]:
train.info()

#### There are 35 columns with between 0 to 10 % rows missing.

In [ ]:
MissingValueTable [(MissingValueTable['Percentages']<=10) & (MissingValueTable['Percentages']>0)].count()

In [ ]:
# MissingValueTable [(MissingValueTable['Percentages']<=10) & (MissingValueTable['Percentages']>0)]

In [ ]:
ColumnsWMissingValue = MissingValueTable [(MissingValueTable['Percentages']<=10) & (MissingValueTable['Percentages']>0)] #['Column Name']

In [ ]:
ColumnsWMissingValue

#### List of Categorical columns with between 0 - 10 percent missing rows.

In [ ]:
ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist()

### Since we cannot impute text at the moment, We are going to remove any rows with missing value in the categorical columns and then take a 1 or 2% sample from the rest of the dataframe.

In [ ]:
train.info()

In [ ]:
train [ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist()].isna().sum()

In [ ]:
#ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist()
train1 = train.dropna (subset = ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist() , axis = 0)

In [ ]:
train1 [ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist()].isna().sum()

In [ ]:
train1 [ColumnsWMissingValue.loc [ColumnsWMissingValue['Data Type'] =='category']['Column Name'].tolist()]

In [ ]:
train2 = train1.sample(frac=0.02,random_state=200) #random state is a seed value

#### I will be imputating Using Multivariate Imputation by Chained Equation.
* Ref: https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779

In [ ]:
train2.info()

In [ ]:
train2.describe()

In [ ]:
ColumnsWMissingValue['Column Name']

In [ ]:
NumColumnWMissingValue1 = train2 [ColumnsWMissingValue['Column Name']].select_dtypes(exclude='category')

In [ ]:
NumColumnWMissingValue1.columns.tolist()

#Tried Multivariate Imputation but found it to be slow 
from impyute.imputation.cs import mice
imputed_training = mice(NumColumnWMissingValue1.values)

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean') 
imp_mean.fit(NumColumnWMissingValue)
imputed_train_df = imp_mean.transform(NumColumnWMissingValue)

dataset = pd.DataFrame(imputed_training)

dataset.columns = NumColumnWMissingValue1.columns.tolist()

dataset.describe()

In [ ]:
NumColumnWMissingValue1.describe()

In [ ]:
BinaryColumns

In [ ]:
for col in NumColumnWMissingValue1.columns.tolist():
    if col in BinaryColumns:
        print (col, 'Binary')
        print (train2[col].mode())
    else:
        
        print ('Not in Binary')

In [ ]:
train2 ['Census_PrimaryDiskTotalCapacity'].mode()

In [ ]:
for col in NumColumnWMissingValue1.columns.tolist():
    if col in BinaryColumns:
        train2[col].fillna(train2[col].mode(), inplace = True)
    else:
        train2[col] =  train2[col].astype (dtype = 'float32')
        train2[col].fillna(train2[col].mean(), inplace = True)

train2['RtpStateBitfield'].fillna(train2['RtpStateBitfield'].mean(), inplace=True)

In [ ]:
train2.describe()

Let digitize the categorical columns with one hot encoder

In [ ]:
train.info()

In [ ]:
train2.head()

In [ ]:
train2.select_dtypes(include='category').columns.tolist()

In [ ]:
train_columns = train2.select_dtypes(include='category').columns.tolist()

In [ ]:
Categorical_Columns = ['ProductName', 'Platform', 'Processor', 'Census_OSArchitecture',
                       'Census_FlightRing','Census_GenuineStateName']

In [ ]:
Drop_columns = [x for x in train_columns if x not in Categorical_Columns]
print (Drop_columns)

In [ ]:
train2 = train2.drop (Drop_columns, axis = 1)

In [ ]:
train2[Categorical_Columns]

In [ ]:
train2[Categorical_Columns] = train2[Categorical_Columns].astype ('category')

In [ ]:
train2[Categorical_Columns].isna().sum()

In [ ]:
Categorical_Columns

In [ ]:
train3 = pd.get_dummies (train2[Categorical_Columns])

In [ ]:
train3

In [ ]:
train2 = train2.drop (Categorical_Columns, axis =1)

In [ ]:
train2.info()

In [ ]:
train4 = pd.concat([train2, train3], axis =1)

In [ ]:
train4.head()

In [49]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train4, test_size=0.3)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [50]:
features = train4.loc[:, train4.columns != 'HasDetections'].columns.tolist()

In [51]:
y_train = train['HasDetections']
X_train = train[features]

In [52]:
y_test = test['HasDetections']
X_test = test[features]

### Ref: https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

In [53]:
import xgboost as xgb

In [54]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

C:\Users\Shahram\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Shahram\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [68]:
param = {
    'eta': 0.3, 
    'max_depth': 10,  
    'objective': 'multi:softprob',  
    'num_class': 5} 

steps = 20  # The number of training iterations

In [69]:
model = xgb.train(param, D_train, steps)

In [70]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.60936054322367
Recall = 0.6076987003749696
Accuracy = 0.6075096817646068


#### Ref: https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 100
params['min_data'] = 50
params['max_depth'] = 100
clf = lgb.train(params, d_train, 100)

In [ ]:
#Prediction
y_pred=clf.predict(X_test)
#convert into binary values
for i in range(0,99):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

### Trying Logisitic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
lr = LogisticRegression(penalty = 'l1', C = 10,random_state = 1)

* Apply logistic regression on the split train/test dataset

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_predict = lr.predict(X_test)

* Compute your model’s accuracy using accuracy_score

In [ ]:
round(accuracy_score(y_test,y_predict),3)